In [1]:
from qTools import *
import Cahit as ck

import scipy.sparse as sp
import numpy as np

from functools import partial
from multiprocessing import Pool, cpu_count

In [2]:
resonatorDimension = 200
g = 1.79
qfreq = 0
resFreq = 2
irreg = False

In [ ]:
def digitalRabi(obj, stepSize):
    sigmaX = sp.kron(QuantumToolbox.operators.identity(obj.subSystems['Cavity1'].dimension), obj.sigmaX(obj.subSystems['Qubit1'].dimension), format='csc')
    HamJC = 2 * np.pi * (((obj.subSystems['Cavity1'].frequency/2)*obj.subSystems['Cavity1'].freeMat) + obj.couplingHam)
    UnitaryJC = QuantumToolbox.liouvillian.Liouvillian(HamJC, timeStep=(stepSize / 2))
    UnitaryAJC = (UnitaryJC @ UnitaryJC)
    Unitary = UnitaryJC @ sigmaX @ UnitaryAJC @ sigmaX @ UnitaryJC
    return Unitary

In [ ]:
qSim = Simulation()
qSim.tProtocol = evolveTimeIndep
JCSys = qSim.qSys

In [ ]:
cav = Cavity(dimension=resonatorDimension, frequency=resFreq)
JCSys.addSubSys(cav)

qub = JCSys.createSubSys(subClass=Qubit)
qub.frequency = qfreq

In [ ]:
JCcoupling = JCSys.JC(cav, qub, g)

JCSys.initialState = sp.kron(states.basis(cav.dimension, 0), states.basis(2, 0), format='csc')

In [ ]:
cavFreqSweep = qSim.addSweep(cav, 'frequency')
cavFreqSweep.sweepList = np.arange(-4, 4, 0.05)

cavParity = parityEXP(cav.freeMat)

In [ ]:
p = Pool(processes=cpu_count())
print('simulating')
statesDigit = qSim.run(p, cav, cavFreqSweep.sweepList)
parityDigit = p.map(partial(expectationKetList, cavParity),statesDigit)
print(JCSys._QuantumSystem__kept)

In [ ]:
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, qSim.times, parityDigit, irregular=irreg)

In [ ]:
JCSys.reset()
JCSys.createSysCoupling([qub,cav],[QuantumToolbox.operators.sigmax, QuantumToolbox.operators.create], g)
JCSys.createSysCoupling([cav, qub],[QuantumToolbox.operators.destroy, QuantumToolbox.operators.sigmax], g)
JCSys.couplingName = 'Rabi'

print(JCSys.couplingName)

In [ ]:
print('simulating')
statesDigit = qSim.run(p, cav, cavFreqSweep.sweepList)
parityDigit = p.map(partial(expectationKetList, cavParity),statesDigit)
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, qSim.times, parityDigit, irregular=irreg)
print(JCSys._QuantumSystem__kept)

In [ ]:
JCSys.reset(to='JC')
#JCSys.addCoupling([qub,cav],[qtbox.operators.destroy, qtbox.operators.create], g)
#JCSys.addCoupling([cav, qub],[qtbox.operators.destroy, qtbox.operators.create], g)
JCSys.sigmaX = QuantumToolbox.operators.sigmax
JCSys.Unitaries = digitalRabi

In [ ]:
print('simulating')
'''for i in range(len(cavFreqSweep.sweepList)):
    qSim.evolveTimeIndep(cav, cavFreqSweep.sweepList[i])'''
statesDigit = qSim.run(p, cav, cavFreqSweep.sweepList)
parityDigit = p.map(partial(expectationKetList, cavParity),statesDigit)
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, qSim.times, parityDigit, irregular=irreg)
print(JCSys._QuantumSystem__kept)

In [ ]:
JCSys.reset(to='Rabi')
print('simulating')
statesDigit = qSim.run(p, cav, cavFreqSweep.sweepList)
parityDigit = p.map(partial(expectationKetList, cavParity),statesDigit)
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, qSim.times, parityDigit, irregular=irreg)
print(JCSys._QuantumSystem__kept)

In [ ]:
JCSys.reset(to='JC')
JCcoupling.couplingStrength = 1
print('simulating')
statesDigit = qSim.run(p, cav, cavFreqSweep.sweepList)
parityDigit = p.map(partial(expectationKetList, cavParity),statesDigit)
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, qSim.times, parityDigit, irregular=irreg)
print(JCSys._QuantumSystem__kept)

In [ ]:
JCSys.reset(to=2)
print(JCSys.couplingName)
JCcoupling.couplingStrength = 1.79
JCSys.constructCompSys()
print('simulating')
statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), cavFreqSweep.sweepList)
parityDigit = p.map(partial(expectationKetList, cavParity),statesDigit)
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, qSim.times, parityDigit, irregular=irreg)
print(JCSys._QuantumSystem__kept)